In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import warnings
# warnings.simplefilter('ignore')

In [ ]:
from urllib.request import urlopen
import json

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from helpers_5a import parse_report, cache_locally, find_coordinates, replace_column_values

In [ ]:
import plotly.graph_objects as go

In [ ]:
mapbox_access_token = open('mapbox_token.txt').read()

---

## Infected map

https://systems.jhu.edu/research/public-health/ncov/ inspired by this dashboard (warning: in the US only city-level counts are mapped, not unassigned locations)

good vid https://www.youtube.com/watch?v=9PYKYjkqnGU

### Extract data from PDF reports

source: [WHO](https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports)

In [ ]:
# Latest reports, that respect follow (more or less) the same format
who_reports = [
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200219-sitrep-30-covid-19.pdf', 4, 5),
    # 31 purposely not included
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200221-sitrep-32-covid-19.pdf', 3, 4),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200222-sitrep-33-covid-19.pdf', 3, 4),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200223-sitrep-34-covid-19.pdf', 2, 3),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200224-sitrep-35-covid-19.pdf', 3, 4),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200225-sitrep-36-covid-19.pdf', 2, 3),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200226-sitrep-37-covid-19.pdf', 2, 3),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200227-sitrep-38-covid-19.pdf', 3, (4,5)),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200228-sitrep-39-covid-19.pdf', 3, (4,5)),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200229-sitrep-40-covid-19.pdf', 3, (4,5)),
    ('https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200301-sitrep-41-covid-19.pdf', 3, (4,5)),
]

Open `helpers_5a.py` for parsing implementation implemented

In [ ]:
%%time
df = pd.concat([parse_report(*args) for args in who_reports])
df

### Make use of pre-cleaned data

In [ ]:
dataset_urls = {
    'cases':      'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv',
    'deaths':     'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv',
    'recovered':  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv',
}

In [ ]:
dfs = {key: pd.read_csv(url) for key, url in dataset_urls.items()}
dfs['cases'];

In [ ]:
region_info_bak = dfs['cases'].iloc[:, :4]

In [ ]:
region_info = dfs['cases'].iloc[:, :4]

In [ ]:
dfs = {key: df.iloc[:, 4:] for key, df in dfs.items()}
dfs['cases'];

---

In [ ]:
region_info = region_info_bak.copy()

In [ ]:
region_info.columns = ['state', 'country', 'lat', 'lon']

# So we can later find the country code
replace_column_values(region_info, 'country', {
    'UK': 'United Kingdom',
    'US': 'United States',
    'Mainland China': 'China',
})

#### Special treatment: Diamond Princess cruise ship

In [ ]:
# Treat the cruise ship as its own country
cruise_row = (region_info.country == 'Others') & (region_info.state == 'Diamond Princess cruise ship')
region_info[cruise_row] = (np.nan, 'Diamond Princess', 33.9593, 138.8067)

In [ ]:
us_mask = region_info.country == 'United States'
region_info.loc[us_mask, 'state'] = region_info[us_mask].state.str.replace(' (From Diamond Princess)', '', regex=False)  # don't care about the source

In [ ]:
aus_cruise_row = (region_info.country == 'Australia') & (region_info.state == 'From Diamond Princess')
region_info.loc[aus_cruise_row, 'state'] = 'Unassigned Location'

#### Sum country totals

In [ ]:
for country, count in region_info.country.value_counts().items():
    mask = region_info.country == country
    
    # For Hong Kong and Macau, which both appear only once as countries but have the subdivision set to Hong Kong, Macau respectively
    if count == 1:
        region_info.loc[mask, 'state'] = np.nan
        
    else:
        print('Adding totals for', country)
        idx = len(region_info)  # append
        region_info.loc[idx] = ('All', country, *find_coordinates(country))

        for df in dfs.values():
            df.loc[idx] = df[mask].sum()

Having `All` in the `state` column means the `country` has multiple states info.

#### Assign standardized country names

In [ ]:
import pycountry

In [ ]:
def find_country_code(country_name):
    if country_name == 'Diamond Princess':
        return 'DP'
    if country_name == 'South Korea':  # Republic of Korea is recognized
        return 'KOR'
    if country_name == 'Macau':  # Macao is recognized though
        return 'MAC'
    if country_name == 'Czech Republic (Czechia)':
        country_name = 'Czech Republic'
    
    try:
        return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except:
        return np.nan

In [ ]:
%%time
region_info['country_code'] = region_info.country.map(find_country_code)

#### Sum state totals

In [ ]:
import re

In [ ]:
def parse_state_info(url, table_number, state_abbrev_pop_columns):
    df = pd.read_html(url)[table_number].iloc[:, state_abbrev_pop_columns]
    df.columns = ['state', 'abbrev', 'population']
    
    for col in ['state', 'abbrev']:
        df[col] = df[col].str.replace(f'\[.*\]', '')  # remove link brackets
        df[col] = df[col].str.replace(f'\[', '')  # remove link brackets
        
    return df

In [ ]:
%%time
state_info = {
    'USA': parse_state_info('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States', 0, [0, 1, 5]),
    'CAN': parse_state_info('https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada',                 1, [1, 2, 6]),
    'AUS': parse_state_info('https://en.wikipedia.org/wiki/States_and_territories_of_Australia',                 2, [1, 4, 6]),
    'CHN': parse_state_info('https://www.worldatlas.com/articles/chinese-provinces-by-population.html',          0, [1, 1, 2]),
}
state_info['CAN'] = state_info['CAN'][:-1]  # has a totals row at the end

In [ ]:
region_info['city'] = np.nan # or subdivision such as Orange County
for i, row in region_info.iterrows():
    if pd.isna(row.state):
        continue
    try:
        city, state = re.match(r'(.+), ([A-Z]{2})$', row.state).groups()
    except AttributeError:
        continue  # no match

    country_states = state_info[row.country_code]
    state_row = country_states[country_states.abbrev == state]
    if len(state_row) == 1:  # if it was an abbreviation
        state = state_row.state.iloc[0]
    print('Assigned', state, 'to', city)

    region_info.loc[i, 'city'] = city
    region_info.loc[i, 'state'] = state

In [ ]:
for state in region_info.state.unique():
    cities_mask = (region_info.state == state) & (~pd.isna(region_info.city))
    if not any(cities_mask):
        continue
    state_row = region_info[cities_mask].iloc[0].copy()  # pick the row for an arbitrary city
    state_row.city = 'All'  # erase the arbitraty city name
    state_row.lat, state_row.lon = find_coordinates(f'{state_row.state}, {state_row.country_code}')  # update coordinates of entire state
    
    idx = len(region_info)
    region_info.loc[idx] = state_row
    
    for df in dfs.values():
        df.loc[idx] = df[cities_mask].sum()

Having `All` in the `city` column means the `state` has data for multiple cities.

#### More specific coordiantes for known regions

In [ ]:
region_info['display_subdivision'] = np.nan
region_info.loc[region_info.state   == 'Hubei', 'display_subdivision'] = 'Wuhan'
region_info.loc[region_info.country == 'Italy', 'display_subdivision'] = 'Milan'

In [ ]:
mask = ~region_info.display_subdivision.isna()
region_info.loc[mask, 'lat'], region_info.loc[mask, 'lon'] = zip(*region_info[mask].display_subdivision.map(find_coordinates))

In [ ]:
for i, row in region_info.iterrows():
    if row.state == 'Unassigned Location':
        region_info.loc[i, 'lat'], region_info.loc[i, 'lon'] = find_coordinates(row.country)
        print('Placed unassigned locations in the middle of', row.country)

#### Add population data

##### Country level

source: https://www.worldometers.info/world-population/population-by-country/

In [ ]:
pop_df = pd.read_html('https://www.worldometers.info/world-population/population-by-country/')[0]
pop_df = pop_df[['Country (or dependency)', 'Population (2020)']].dropna()
pop_df.columns = ['country', 'population']
pop_df['country_code'] = pop_df.country.map(find_country_code)

When fuzzy searching both Netherlands (population 17M), and Caribbean Netherlands (population 26k) correctly match NLD. But we will assume it is talking about the most populous region. Same for France and Reunion, and other such territories.

In [ ]:
pop_df = pop_df\
    .drop('country', axis=1)\
    .sort_values(by='population', ascending=False)\
    .drop_duplicates(subset='country_code', keep='first')

source: https://en.wikipedia.org/wiki/Diamond_Princess_(ship)

In [ ]:
pop_df.loc[len(pop_df)] = (2670 + 1100, 'DP')

In [ ]:
region_info = pd.merge(region_info, pop_df, on='country_code', how='left')

##### State level

We can do this trick because no two countries in this list have the same state names:

In [ ]:
all_states_info = pd.concat(state_info.values())
assert len(all_states_info) == sum(map(len, state_info.values()))

In [ ]:
state_pop_series = pd.merge(region_info.state, all_states_info.drop('abbrev', axis=1), on='state', how='left').population
mask = ~pd.isna(state_pop_series)  # state field is filled and the state exists in the population data (including != All)
region_info.loc[mask, 'population'] = state_pop_series[mask]

##### City level

Note: could go down to city level (eg: using data from [here](https://www.worldometers.info/world-population/us-population/)) but in the US at least, the data provided sometimes comes at a county level.

In [ ]:
# cities_mask = ~pd.isna(region_info.city) & (region_info.city != 'All')
# region_info.loc[cities_mask, 'population'] = np.nan

---

Sort:

In [ ]:
region_info.sort_values(by=['country', 'state', 'city'], inplace=True)
sort_indices = region_info.index

for key, df in dfs.items():
    dfs[key] = df.iloc[sort_indices].reset_index()
    
region_info.reset_index(inplace=True)

---

In [ ]:
pd.options.display.float_format = '{:,.10g}'.format
pd.options.display.max_rows = 200

In [ ]:
region_info = region_info[['country', 'country_code', 'state', 'city', 'display_subdivision', 'lat', 'lon', 'population']]
region_info

---

In [ ]:
countries_mask = region_info.state.isin([np.nan, 'All'])  # includes country "Diamond Princess"
cities_mask = ~pd.isna(region_info.city) & (region_info.city != 'All')
states_mask = ~countries_mask & ~cities_mask  # includes "state" Unassigned Location

### Get region boundaries

#### Countries

source: naturalearthdata.com

In [ ]:
%%time
with urlopen('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_10m_admin_0_countries.geojson') as response:
    countries_geojson = json.load(response)

In [ ]:
for country_shape in countries_geojson['features']:
    country_shape['id'] = country_shape['properties']['ADM0_A3']

drew on geojson.io

In [ ]:
diamond_princess_shape = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          138.80401611328125,
          33.988918483762156
        ],
        [
          138.77105712890625,
          33.97297577172598
        ],
        [
          138.7738037109375,
          33.94335994657882
        ],
        [
          138.7957763671875,
          33.92740869431181
        ],
        [
          138.82049560546875,
          33.93196649986436
        ],
        [
          138.83697509765625,
          33.950195282756994
        ],
        [
          138.83697509765625,
          33.97525348507592
        ],
        [
          138.80401611328125,
          33.988918483762156
        ]
      ]
    ]
  }
}
    
diamond_princess_shape['id'] = 'DP'

In [ ]:
countries_geojson['features'].append(diamond_princess_shape)

In [ ]:
countries_with_shape = {shape['id'] for shape in countries_geojson['features']}
print('Countries with missing shape:', set(region_info[countries_mask].country_code) - countries_with_shape)

#### States/provinces

maps borders are not definitive: https://www.youtube.com/watch?v=q9ZMub2UrKU

In [ ]:
%%time
with urlopen('https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_10m_admin_1_states_provinces.geojson') as response:
    states_geojson = json.load(response)

In [ ]:
from unidecode import unidecode

In [ ]:
# There are very many shapes, so just keep the ones that we'll work with
countries_with_states = region_info[states_mask].country_code.unique().tolist()
states_geojson['features'] = [shape 
                              for shape in states_geojson['features'] 
                              if shape['properties']['adm0_a3'] in countries_with_states]

In [ ]:
for state_shape in states_geojson['features']:
    id_ = state_shape['properties']['woe_name'] or state_shape['properties']['name']
    id_ = unidecode(id_)  # Québec -> Quebec
    
    if id_ == 'Inner Mongol':
        id_ = 'Inner Mongolia'
    if id_ == 'Xizang':
        id_ = 'Tibet'
    
    state_shape['id'] = id_

In [ ]:
states_with_shape = {shape['id'] for shape in states_geojson['features']}
print('States with missing shape:', set(region_info[states_mask].state) - states_with_shape)

### Hover text

In [ ]:
def rbn(x, precision=1, threshold=.9):
    """ readable big number """
    was_negative = x < 0
    x = abs(x)
    
    threshold = threshold or 1
    if x < 1_000 * threshold:
        suffix = ''
    elif x < 1_000_000 * threshold:
        x /= 1_000.
        suffix = 'k'
    elif x < 1_000_000_000 * threshold:
        x /= 1_000_000.
        suffix = 'm'
    else:
        x /= 1_000_000_000.
        suffix = 'b'
        
    fmt = '{:.' + str(precision) + 'f}'
    nr = fmt.format(x)
    
    nr = nr.lstrip('0')  # remove leading zeros
    
    if precision > 0: # otherwise we strip useful zeros
        nr = nr.rstrip('0').rstrip('.')  # remove trailing zeros

    if nr == '':
        return '0'
    
    preffix = '-' if was_negative else ''
    return preffix + nr + suffix

In [ ]:
def rsn(x, percentage=True):
    """ readable small number """
    fmt_type = '%' if percentage else 'f'
    if x > .01:
        precision = 1
    else:
        precision = 3
    fmt = '{:.' + str(precision) + fmt_type +'}'
    
    nr = fmt.format(x)
    #nr = nr.lstrip('0')  # remove leading zeros
    
    nr = nr.rstrip('%')
    nr = nr.rstrip('0').rstrip('.')  # remove trailing zeros
    if percentage:
        nr += '%'
    
    if nr.replace('%', '') == '':
        return '0'
    return nr

---

In [ ]:
def build_hover_text(idx, row):
    population_label = 'Population' if pd.isna(row.city) else 'State population'
    
    cases  = dfs['cases']    .iloc[idx, -1]
    deaths = dfs['deaths']   .iloc[idx, -1]
    recvd  = dfs['recovered'].iloc[idx, -1]
    
    cases_per = cases / row.population
    cases_details = 'less than 0.001%' if cases_per * 100 < 0.001 else f'{rsn(cases_per, 4)}'
    
    deaths_paren = '' if deaths == 0 else f' ({rsn(deaths/cases)} of cases)'
    recvd_paren  = '' if recvd  == 0 else f' ({rsn(recvd /cases)} of cases)'
    
    title = row.country
    if not pd.isna(row.state) and row.state != 'All':
        title = f'{row.state}, {title}'
    if not pd.isna(row.city) and row.city != 'All':
        title = f'{row.city}, {title}'
    
    s = (f'<b>{title}</b><br>'
         f'{population_label}: {rbn(row.population)}<br>'
         f'Cases: {cases:,} ({cases_details} of population)<br>'
         f'Deaths: {deaths:,}{deaths_paren}<br>'
         f'Recovered: {recvd:,}{recvd_paren}<br>'
        )
    return s

In [ ]:
region_info['hover_text'] = [build_hover_text(*t) for t in region_info.iterrows()]

### Preprocess for density map

Ideally (also impossibly), we would have the coordinates of each case/death, and plot them individually (achieveing a granularity level similar to [this](https://docs.mapbox.com/help/tutorials/make-a-heatmap-with-mapbox-gl-js/)). Instead, we have counts aggregated at various region levels. We will add jitter (random noise) to the default location assigned to each region (center of city/county, or state/province or country), as to simulate granularity.

#### Add jitter

A Density map displays four (implicit) axes: vertical and horizontal axes, amount of points, and also magnitude of each point. In our case the magnitude will represent the mortality rate, also known as the case fatality rate (CFR).

In [ ]:
countries_without_states_mask = countries_mask & (~region_info.country_code.isin(countries_with_states))

In [ ]:
states_without_cities_mask = states_mask & pd.isna(region_info.city)

---

In [ ]:
jitter_mask = (countries_without_states_mask | states_without_cities_mask) & (dfs['cases'].iloc[:, -1] > 9)

In [ ]:
jitter = .2  # 1 unit degree of latitude is about 110km/69 miles (differs by latitude)
generate_jitter = lambda mean, count: np.random.normal(mean, jitter, count)

In [ ]:
get_deaths = lambda i: dfs['deaths'].iloc[i, -1]

In [ ]:
jittered = pd.concat(
    pd.DataFrame({
        'lat': generate_jitter(row.lat, get_deaths(i)),
        'lon': generate_jitter(row.lon, get_deaths(i)),
        'z': get_deaths(i) / dfs['cases'].iloc[i, -1],
        'hover_text': row.hover_text
    })
    for i, row in region_info[jitter_mask].iterrows()
    if get_deaths(i) > 0
)

In [ ]:
jittered.z.describe()

### Plot

In [ ]:
# Log scale for heatmap is not supported so we create it manually
zmax_prelog = 10_000
zmax = np.log10(zmax_prelog)

In [ ]:
tickvals = np.arange(0, zmax+.001, 1)
ticktext = 10 ** tickvals
ticktext = [f'{v:,.0f}' for v in ticktext]

ticktext[0] = '1'
ticktext[-1] += '+'

In [ ]:
choro_options = dict(
    marker_line=dict(
        color='rgb(92, 4, 18)',  # brownish red
        width=.2,
    ),
    
    colorscale='Reds',
    zauto=False,
    zmin=1,
    zmax=zmax,
    colorbar=dict(
        tickvals=tickvals,
        ticktext=ticktext,
        title='Cases',
        
        # Possible option to prevent map resizing
#         xanchor='left',
#         x=0,
#         tickfont_color='white',
    ),
    
    hoverinfo='text',
)

fig = go.Figure([
    # Cities scatter
    go.Scattermapbox(
        below='',
        lat=region_info[cities_mask].lat,
        lon=region_info[cities_mask].lon,
        
        name='',
        text=region_info[cities_mask].hover_text,
        hoverinfo='text',

        marker=dict(
            color='rgb(92, 4, 18)',  # brownish red
        )
    ),

    # Cases choropleth
    go.Choroplethmapbox(
        geojson=countries_geojson, 
        locations=region_info[countries_without_states_mask].country_code, 
        z=np.log10(dfs['cases'][countries_without_states_mask].iloc[:, -1]),
        text=region_info[countries_without_states_mask].hover_text,

        **choro_options,
    ),
    go.Choroplethmapbox(
        geojson=states_geojson, 
        locations=region_info[states_mask].state, 
        z=np.log10(dfs['cases'][states_mask].iloc[:, -1]),
        text=region_info[states_mask].hover_text,
        
        **choro_options,
        showscale=False,
    ),
    
    # Mortality density box
    go.Densitymapbox(
        visible=False,
        below='',  # place it above everything
        
        lat=jittered.lat, 
        lon=jittered.lon, 
        z=jittered.z,
        text=jittered.hover_text,
        
        name='',
        hovertemplate='%{text}',

        radius=10,
        colorscale='Plasma',

        zauto=False, zmin=0,
        
        colorbar=dict(
            title='Mortality',
            tickformat='%.%'
        )
    ),


])

fig.update_layout(
    margin=dict(t=0, b=0, r=0, l=0),
    height=800,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style='light',  # carto-positron also works well and requires no key
    ),
    
    
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(
                    args=[dict(visible=[True, True, True, False])],
                    label='Cases',
                    method='update'
                ),
                dict(
                    args=[dict(visible=[True, False, False, True])],
                    label='Mortality',
                    method='update'
                )
            ],

            pad=dict(t=10),
            direction='left',

            x=0,
            xanchor='left',

            y=1.07,
            yanchor='top',
        )
    ],
)

TODO: 
 - slider & animation over time
 - per country bar plot to the side with time increments

Note: division by zero happens when the cases count is zero. The data contains zeros because at the requested date, there are zero cases. The resulting behavior is correct: a `NaN` value is not plotted.

At higher zoom levels, the intensity of the color indicates the (absolute) amount of deaths.

---

Caveat: numbers are not precise, especially in China, which changed the definition of a "confirmed" case several times [source](https://www.nytimes.com/interactive/2020/world/coronavirus-maps.html)